In [1]:
%cd ../scripts 
from address import dist_between_two_lat_lon, find_closest_location
import pandas as pd
import geopandas as gpd
from os.path import exists
import warnings

/mnt/c/Users/ohanr/OneDrive/Unimelb/3rd Year/MAST30034/generic-real-estate-consulting-project-group-25/scripts


In [ ]:
listing = pd.read_csv("../data/curated/Distances_pre/listings_w_cords.csv")
listing["ClosestDstToStation"] = None
listing["ClosestStation"] = None
listing["ClosestDstToShoppingCentre"] = None
listing["ClosestShop"] = None
listing["ClosestDstToUni"] = None
listing["ClosestUni"] = None
if exists("../data/curated/listings_with_distances.csv"):
    listing = pd.read_csv("../data/curated/listings_with_distances.csv")

With the following code blocks, often times the server would crash in the local machine due to device's RAM limitations, hence the code needed to be re-ran accrodingly depends on which row of data the server crashed on

## Distance to closest shopping centre

In [ ]:
shops = pd.read_csv("../data/raw/ShoppingCentreData.csv")

In [ ]:
saved_index = 0
for i in range(saved_index, len(listing)):
    try: 
        lng = listing["Longitude"][i] 
        lat = listing["Latitude"][i]
    except KeyError:
        continue
    d, clo = find_closest_location(lat, lng, shops, "Latitude", "Longitude", "Shopping Centre Name")
    listing["ClosestDstToShoppingCentre"][i] = d
    listing["ClosestShop"][i] = clo
    ## Save in partitions to compensate unstable api connection due to limited RAM
    if (i != saved_index) and (i%50 == 0):
        listing.to_csv("../data/curated/listings_with_distances.csv", index=False)
        saved_index = i
        print(saved_index)
listing.to_csv("../data/curated/listings_with_distances.csv", index=False)   

: 

## Distance to CBD

In [ ]:
CBD = (-37.8136, 144.9631)
dst = []
for i in range(0, len(listing)):
    lng = listing["Longitude"][i] 
    lat = listing["Latitude"][i]
    d = dist_between_two_lat_lon(lat, CBD[0], lng, CBD[1])
    dst.append(d)



In [ ]:
listing["DstToCBD"] = dst
listing.to_csv("../data/curated/listings_w_distance.csv", index = False)

## Distance to Nearest Train Station

In [ ]:
ptv = gpd.read_file("../data/raw/PTV/PTV_METRO_TRAIN_STATION.shp")
vline = gpd.read_file("../data/raw/PTV_Regional/PTV_REGIONAL_TRAIN_STATION.shp")
stations = gpd.GeoDataFrame(pd.concat([ptv, vline]))
listing = pd.read_csv("../data/curated/listings_with_distances.csv")

In [ ]:
saved_index = 0
for i in range(saved_index, len(listing)):
    try: 
        lng = listing["Longitude"][i] 
        lat = listing["Latitude"][i]
    except KeyError:
        continue
    d, clo = find_closest_location(lat, lng, stations, "LATITUDE", "LONGITUDE", "STOP_NAME")
    listing["ClosestDstToStation"][i] = d
    listing["ClosestStation"][i] = clo
    ## Save in partitions to compensate unstable api connection due to limited RAM
    if (i != saved_index) and (i%50 == 0):
        listing.to_csv("../data/curated/listings_with_distances.csv", index = False)
        saved_index = i
        print(saved_index)
listing.to_csv("../data/curated/listings_with_distances.csv", index=False)   

## Distance to Universities

In [ ]:
universities = pd.read_csv("../data/curated/universities.csv")

In [ ]:

saved_index = 300
for i in range(saved_index, len(listing)):
    try: 
        lng = listing["Longitude"][i] 
        lat = listing["Latitude"][i]
    except KeyError:
        continue
    d, clo = find_closest_location(lat, lng, universities, "Latitude", "Longitude", "University")
    listing["ClosestDstToUni"][i] = d
    listing["ClosestUni"][i] = clo
    ## Save in partitions to compensate unstable api connection due to limited RAM
    if (i != saved_index) and (i%50 == 0):
        listing.to_csv("../data/curated/listings_with_distances.csv", index = False)
        saved_index = i
        print(saved_index)
listing.to_csv("../data/curated/listings_with_distances.csv", index=False)   

## Suburb to Nearest Station Distance

In [ ]:
ptv = gpd.read_file("../data/raw/PTV/PTV_METRO_TRAIN_STATION.shp")
vline = gpd.read_file("../data/raw/PTV_Regional/PTV_REGIONAL_TRAIN_STATION.shp")
stations = gpd.GeoDataFrame(pd.concat([ptv, vline]))
suburbs = pd.read_csv("../data/raw/sub_4_pred_done.csv")

In [ ]:
suburbs["DstToStation"] = None
suburbs["ClosestStation"] = None

In [ ]:
saved_index = 0
for i in range(saved_index, len(suburbs)):
    try: 
        lng = suburbs["Longitude"][i] 
        lat = suburbs["Latitude"][i]
    except KeyError:
        continue
    d, clo = find_closest_location(lat, lng, stations, "LATITUDE", "LONGITUDE", "STOP_NAME")
    suburbs["DstToStation"][i] = d
    suburbs["ClosestStation"][i] = clo
    ## Save in partitions to compensate unstable api connection due to limited RAM
    if (i != saved_index) and (i%50 == 0):
        suburbs.to_csv("../data/curated/suburbs_w_dstTrain.csv", index = False)
        saved_index = i
        print(saved_index)
suburbs.to_csv("../data/curated/suburbs_w_dstTrain.csv", index=False)   